In [5]:
from preprocess import cargar_y_preprocesar_datos
from utils import print_score
from sklearn.ensemble import RandomForestClassifier

In [6]:
ruta = "./data/credit_card_fraud_dataset.csv"
X_train, X_val, X_test, y_train, y_val, y_test = cargar_y_preprocesar_datos(ruta)

In [7]:
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)
print_score(modelo, X_train, y_train, X_val, y_val, train=True)
print_score(modelo, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 99.99%
_______________________________________________
CLASSIFICATION REPORT:
                      0           1  accuracy     macro avg  weighted avg
precision      0.999933    1.000000  0.999933      0.999966      0.999933
recall         1.000000    0.993333  0.999933      0.996667      0.999933
f1-score       0.999966    0.996656  0.999933      0.998311      0.999933
support    59400.000000  600.000000  0.999933  60000.000000  60000.000000
_______________________________________________
Confusion Matrix: 
 [[59400     0]
 [    4   596]]

Test Result:
Accuracy Score: 99.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0      1  accuracy     macro avg  weighted avg
precision      0.989999    0.0   0.98995      0.495000       0.98010
recall         0.999949    0.0   0.98995      0.499975       0.98995
f1-score       0.994950    0.0   0.98995      0.497475       0.98500
support    19800.000000  200.0   0.9899

El modelo tiene un 99.99%  de accuracy en el entrenamiento y clasifica correctanmente todos los verdaderos negativos y solo se equivoca en 4 transacciones fraudulentas pero en validación funciona mal, no clasifica ningún verdadero positivo, lo que quiere decir que mi modelo esta memorizando los datos de entrenamiento, es decir, tiene sobreajuste.

El modelo tiene una exactitud global muy alta, pero hay que tener en cuenta que se debe sobre todo a que las transacciones normales tienen mucha má frecuencia que las fraudulentas y tenemos un dataset desbalanceado en favor de la clase 0 que es la mayoritaria. Hay que tener en cuenta que en el contexto de detección de fraude lo que nos interesa es detectar aquellas transacciones que son fraudulentas pero que no han sido detectadas, es decir, los falsos negativos para la clase 1, que como hemos visto en la correspondiente métrica de recall supone un 26% del total de transacciones categorizadas como fraudulentas.

In [14]:
modelo_mejorado = RandomForestClassifier(n_estimators=100,  max_depth=10, min_samples_leaf=5, min_samples_split=10, random_state=42)
modelo_mejorado.fit(X_train, y_train)

print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_mejorado, X_train, y_train, X_val, y_val, train=False)

c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Train Result:
Accuracy Score: 99.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0      1  accuracy     macro avg  weighted avg
precision      0.990000    0.0      0.99      0.495000      0.980100
recall         1.000000    0.0      0.99      0.500000      0.990000
f1-score       0.994975    0.0      0.99      0.497487      0.985025
support    59400.000000  600.0      0.99  60000.000000  60000.000000
_______________________________________________
Confusion Matrix: 
 [[59400     0]
 [  600     0]]

Test Result:
Accuracy Score: 99.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0      1  accuracy     macro avg  weighted avg
precision      0.990000    0.0      0.99      0.495000      0.980100
recall         1.000000    0.0      0.99      0.500000      0.990000
f1-score       0.994975    0.0      0.99      0.497487      0.985025
support    19800.000000  200.0      0.99  20000.000000  20000.00

c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Con este nuevo modelo se ha solucionado el sobreajuste, antes el El rendimiento en entrenamiento era 99.99% y en validación 99.00%, ahora el  rendimiento en entrenamiento es 99.00% y en validación también es 99.00%, el modelo ya no está memorizando, ha aprendido un patrón general y lo aplica tanto a los datos que ha visto como a los que no.

Si se observa la matriz de confusión tanto en entrenamiento como en validación, el modelo ha aprendido que la mejor estrategia para maximizar la accuracy es ignorar por completo la clase 1, no ha sido capaz de encontrar un patrón  en los 600 ejemplos de fraude como para justificar el riesgo de una predicción incorrecta.

La regularización por sí sola no es suficiente porque hay un gran desbalance en las clases. 

**Aplicar sobremuestreo con SMOTE**

Esta técnica `balanceará` el set de entrenamiento creando nuevos ejemplos sintéticos de fraude, lo cual le dará al modelo muchos más ejemplos de los que aprender.

In [16]:
from imblearn.over_sampling import SMOTE

#Aplicar SMOTE SOLO al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#Usar modelo regularizado 
#Ya sabemos que este modelo no sobreajusta, así que es perfecto para usarlo con los nuevos datos
modelo_final = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=5, min_samples_split=10, random_state=42)

#Entrenar con los datos REMUESTREADOS
modelo_final.fit(X_train_resampled, y_train_resampled)

#Evaluar en los datos de validación ORIGINALES (desbalanceados)
print_score(modelo_final, X_train_resampled, y_train_resampled, X_val, y_val, train=True)
print_score(modelo_final, X_train_resampled, y_train_resampled, X_val, y_val, train=False)

Train Result:
Accuracy Score: 85.51%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.846829      0.863671  0.855051       0.855250       0.855250
recall         0.866902      0.843199  0.855051       0.855051       0.855051
f1-score       0.856748      0.853312  0.855051       0.855030       0.855030
support    59400.000000  59400.000000  0.855051  118800.000000  118800.000000
_______________________________________________
Confusion Matrix: 
 [[51494  7906]
 [ 9314 50086]]


--- Resultados del Modelo con SMOTE en el set de Validación ---
Test Result:
Accuracy Score: 85.95%
_______________________________________________
CLASSIFICATION REPORT:
                      0           1  accuracy     macro avg  weighted avg
precision      0.989964    0.009763   0.85945      0.499864      0.980162
recall         0.866818    0.130000   0.85945      0.498409      0.859450
f1-score

Objetivo: aumentar la Precisión sin sacrificar demasiado el Recall, se quieren menos falsas alarmas.

- Ajuste de hiperparámetros: usar `GridSearchCV` para encontrar una mejor combinación.
    - Buscar un balance: aumentar max_depth un poco (ej. 12, 15) para que el modelo pueda aprender reglas más complejas y ser más preciso.
    - Aumentar min_samples_leaf (ej. 10, 20) para que el modelo sea más conservador y genere menos falsos positivos.

- Ajustar el Umbral de Decisión:
    - Por defecto `predict()` usa un umbral de 0.5, si la probabilidad de fraude es > 0.5, lo clasifica como fraude.
    - Dado que se tienen muchos `Falsos Positivos`, significa que muchas transacciones legítimas están obteniendo una puntuación de fraude de, por ej. 0.55 o 0.6.
    - Subir el umbral: En lugar de 0.5, prueba a clasificar como fraude solo si la probabilidad es > 0.8 o 0.9. Esto reducirá drásticamente los Falsos Positivos (aumentando la Precisión), pero probablemente también reducirá un poco los Verdaderos Positivos (bajando el Recall). Tienes que encontrar el punto dulce.

In [20]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_estimators=100, random_state=42)

param_grid = {
    'max_depth': [10, 20, None],          # Profundidad del árbol. None = sin límite.
    'min_samples_leaf': [2, 5, 10],       # Mínimo de muestras en una hoja.
    'min_samples_split': [5, 10, 20],     # Mínimo para dividir un nodo.
    'criterion': ['gini', 'entropy']      # Criterio para medir la calidad de una división.
}

# Elegir la métrica de evaluación. 'f1' es una excelente opción de balance.
# Para datasets desbalanceados, siempre especificar la clase positiva: 'f1_weighted' o 'roc_auc' son buenas
scoring_metric =  'f1_macro'

# Configurar GridSearchCV
# cv=3 significa 3-fold cross-validation. Para cada una de las 54 combinaciones,
# entrenará y validará el modelo 3 veces. Total de entrenamientos: 54 * 3 = 162.
grid_search = GridSearchCV(estimator=rf, 
                            param_grid=param_grid, 
                            cv=3, 
                            scoring=scoring_metric,
                            verbose=2) # verbose=2 te mostrará el progreso

# --- PASO 3: Ejecutar la búsqueda ---
print(f"\nIniciando GridSearchCV... Probando {len(param_grid['max_depth'])*len(param_grid['min_samples_leaf'])*len(param_grid['min_samples_split'])*len(param_grid['criterion'])} combinaciones.")
# Se entrena sobre los datos remuestreados
grid_search.fit(X_train_resampled, y_train_resampled)

# --- PASO 4: Analizar los resultados ---
print("\n--- Resultados de GridSearchCV ---")
print(f"Mejor puntuación ({scoring_metric}): {grid_search.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# El objeto `grid_search` se re-entrena automáticamente con los mejores parámetros
# sobre TODO el conjunto de datos que le pasaste (X_train_resampled).
# Por lo tanto, `grid_search.best_estimator_` es tu modelo final y optimizado.
best_rf_model = grid_search.best_estimator_

# --- PASO 5: Evaluar el modelo optimizado en el set de validación ---
print("\n--- Resultados del Modelo OPTIMIZADO en el set de Validación ---")
# Usamos el set de validación original, que el modelo nunca ha visto.
print_score(best_rf_model, X_train_resampled, y_train_resampled, X_val, y_val, train=False)



Iniciando GridSearchCV... Probando 54 combinaciones.
Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=  11.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=   9.3s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=5; total time=  11.5s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=  11.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=   9.9s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=10; total time=  10.2s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total time=  10.6s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total time=  12.0s
[CV] END criterion=gini, max_depth=10, min_samples_leaf=2, min_samples_split=20; total 